<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e270c6777d546843956aef77ce1c9b7ed001f7b03490309cf7735202e8ea64d6
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 15:17:37
-------------------
qualified stocks: 90
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.00 L
Current:  1.40 C
-------------------
Today PnL: 77.92 K (0.56%)
Current PnL: -17.72 L (-11.92%)
CY Booked + Current PnL: -8.15 L (-5.48%)
-------------------
Total profit:  3.13 L
Total loss:  -20.85 L
-------------------
Total Booked + Current PnL: 18.83 L (15.47%)
Total Booked PnL: 36.55 L (30.03%)
Curr Year Booked PnL: 9.57 L (6.83%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.76 L (56.24%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.89%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-28.97,70.0,H-MC,6.83,142120.0,27184.0,9394.0,1.21,23.65,6.61,31.83,81.0,2.89,1.05,47.31,X40N,ATH,IT
80,TTKPRESTIG,770.00,107.47,71.0,M-SC,6.97,89349.0,-11428.0,11517.0,0.02,-11.34,12.89,0.09,245.0,-0.99,0.66,17.38,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.86,58.0,H-LC,8.94,170225.0,14048.0,12052.0,0.72,8.99,7.08,16.71,22.0,1.17,1.26,28.00,X40N,BTT,HEALTHCARE
65,SBIN,863.00,-13.25,51.0,M-LC,4.45,215246.0,13766.0,13453.0,1.04,6.83,6.25,13.51,60.0,1.02,1.59,20.29,XY25,NTT,BANKS
15,BRITANNIA,6362.67,17.43,64.0,H-LC,9.14,212598.0,14502.0,16455.0,0.24,7.32,7.74,15.63,26.0,0.88,1.57,30.77,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.94,62.0,M-MC,7.61,223688.0,-1274.0,168929.0,5.53,-0.57,75.52,74.53,192.0,-0.01,1.66,29.87,XY24,BTT,STEEL
48,KPIGREEN,731.35,4.80,44.0,H-SC,11.08,125093.0,-204.0,59207.0,0.88,-0.16,47.33,47.09,141.0,-0.00,0.93,56.56,MH,ATH,POWER
54,PGHH,18062.58,-30.15,56.0,H-MC,5.36,202350.0,1530.0,68597.0,-0.85,0.76,33.90,34.92,80.0,0.02,1.50,5.47,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,1.08,42.0,H-LC,9.96,230382.0,-11542.0,74805.0,0.12,-4.77,32.47,26.15,8.0,-0.15,1.71,8.87,X40,ATH,IT
59,RELIANCE,1533.00,-14.41,43.0,H-LC,7.66,214219.0,3853.0,24935.0,0.49,1.83,11.64,13.68,37.0,0.15,1.59,18.66,XY25,NTT,REFINERIES
46,JIOFIN,387.00,-0.58,44.0,H-LC,14.32,219800.0,2394.0,51104.0,0.72,1.10,23.25,24.61,48.0,0.05,1.63,56.52,XY24,BTT,FINANCE
42,INFY,2275.00,-19.24,46.0,H-LC,11.29,315091.0,2203.0,169487.0,-1.29,0.70,53.79,54.87,3.0,0.01,2.33,7.40,X40,BTT,IT
77,TCS,4998.00,-26.76,48.0,H-LC,14.72,291391.0,-54565.0,178419.0,-0.37,-15.77,61.23,35.80,1.0,-0.31,2.16,3.23,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-5.78,52.0,H-LC,2.37,233348.0,-12803.0,92009.0,-1.11,-5.20,39.43,32.18,16.0,-0.14,1.73,28.97,X200,ATH,IT
53,NESTLEIND,1377.00,-9.11,60.0,H-LC,3.10,279630.0,14204.0,42588.0,-0.49,5.35,15.23,21.40,11.0,0.33,2.07,13.55,XY25,NTT,FMCG
44,ITC,452.00,-37.50,54.0,H-LC,3.25,201444.0,1306.0,19580.0,1.28,0.65,9.72,10.44,4.0,0.07,1.49,6.25,X40,NTT,FMCG
20,CIPLA,1795.00,-18.75,58.0,H-LC,5.09,216392.0,11892.0,29516.0,0.66,5.82,13.64,20.25,10.0,0.40,1.60,15.06,X40N,BTT,PHARMA
86,VBL,671.64,-10.56,53.0,H-LC,7.46,322254.0,6412.0,106247.0,1.18,2.03,32.97,35.67,5.0,0.06,2.39,16.17,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.94,62.0,M-MC,7.61,223688.0,-1274.0,168929.0,5.53,-0.57,75.52,74.53,192.0,-0.01,1.66,29.87,XY24,BTT,STEEL
64,SATIN,274.00,-27.73,56.0,H-SC,8.27,143667.0,-52307.0,125120.0,4.27,-26.69,87.09,37.16,126.0,-0.42,1.06,9.23,XY24,NTT,FINANCE
8,AWL,485.00,-64.07,61.0,H-MC,6.29,220729.0,-47718.0,179386.0,3.94,-17.78,81.27,49.05,116.0,-0.27,1.63,13.49,XY24,NTT,FMCG
52,NATIONALUM,247.44,-45.32,71.0,H-MC,0.47,109816.0,9312.0,21326.0,3.33,9.27,19.42,30.49,79.0,0.44,0.81,44.67,MH,ATH,METALS
58,RELAXO,1176.00,-30.97,63.0,H-SC,6.30,64026.0,-51512.0,86506.0,2.80,-44.58,135.11,30.28,136.0,-0.60,0.47,25.87,X40N,NTT,FOOTWEAR


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HONAUT,58357.33,-21.65,39.0,H-SC,3.60,110385.0,-17553.0,64686.0,-2.30,-13.72,58.60,36.84,143.0,-0.27,0.82,13.01,X40N,ATH,ELECTRICAL
66,SFL,1287.00,42.17,61.0,M-SC,8.95,187811.0,-74407.0,144239.0,-1.54,-28.38,76.80,26.63,229.0,-0.52,1.39,16.77,XY24,NTT,MISC
23,COLPAL,3767.14,2.04,60.0,H-MC,9.58,185734.0,-26007.0,108097.0,-1.35,-12.28,58.20,38.77,84.0,-0.24,1.38,10.55,XY25,ATH,FMCG
42,INFY,2275.00,-19.24,46.0,H-LC,11.29,315091.0,2203.0,169487.0,-1.29,0.70,53.79,54.87,3.0,0.01,2.33,7.40,X40,BTT,IT
50,LTIM,7230.20,-5.78,52.0,H-LC,2.37,233348.0,-12803.0,92009.0,-1.11,-5.20,39.43,32.18,16.0,-0.14,1.73,28.97,X200,ATH,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TTKPRESTIG,770.0,107.47,71.0,M-SC,6.97,89349.0,-11428.0,11517.0,0.02,-11.34,12.89,0.09,245.0,-0.99,0.66,17.38,OX40N,NTT,DURABLES
47,KANSAINER,340.0,-68.16,56.0,H-SC,3.69,219897.0,-49770.0,86112.0,0.97,-18.46,39.16,13.47,138.0,-0.58,1.63,11.22,XY24,NTT,PAINTS
81,UJJIVANSFB,60.0,84.36,44.0,H-SC,13.81,116181.0,-26298.0,45822.0,-0.09,-18.46,39.44,13.70,163.0,-0.57,0.86,37.13,OX40N,NTT,BANKS
40,INDIGOPNTS,1408.0,116.52,52.0,M-SC,3.04,141968.0,-32591.0,32624.0,2.16,-18.67,22.98,0.02,221.0,-1.00,1.05,23.49,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.29,43.0,H-SC,3.09,113571.0,-30232.0,56979.0,-0.37,-21.02,50.17,18.60,149.0,-0.53,0.84,25.33,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-26.76,48.0,H-LC,14.72,291391.0,-54565.0,178419.0,-0.37,-15.77,61.23,35.80,1.0,-0.31,2.16,3.23,X40,BTT,IT
42,INFY,2275.00,-19.24,46.0,H-LC,11.29,315091.0,2203.0,169487.0,-1.29,0.70,53.79,54.87,3.0,0.01,2.33,7.40,X40,BTT,IT
44,ITC,452.00,-37.50,54.0,H-LC,3.25,201444.0,1306.0,19580.0,1.28,0.65,9.72,10.44,4.0,0.07,1.49,6.25,X40,NTT,FMCG
86,VBL,671.64,-10.56,53.0,H-LC,7.46,322254.0,6412.0,106247.0,1.18,2.03,32.97,35.67,5.0,0.06,2.39,16.17,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.42,248971.0,-12648.0,131855.0,0.75,-4.83,52.96,45.57,7.0,-0.10,1.84,5.94,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.45,187215.0,36302.0,21006.0,0.70,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
26,DIXON,18940.15,-28.97,70.0,H-MC,6.83,142120.0,27184.0,9394.0,1.21,23.65,6.61,31.83,81.0,2.89,1.05,47.31,X40N,ATH,IT
27,DMART,5201.00,-6.62,70.0,H-LC,14.38,233225.0,48744.0,21620.0,1.70,26.42,9.27,38.14,38.0,2.25,1.73,39.83,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,13.61,67.0,H-MC,3.24,196240.0,31570.0,36403.0,1.14,19.17,18.55,41.28,89.0,0.87,1.45,28.68,X40N,ATH,AC
87,VOLTAS,1918.49,2.56,68.0,H-MC,4.94,215235.0,23493.0,72534.0,-0.40,12.25,33.70,50.08,99.0,0.32,1.59,19.88,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.45,187215.0,36302.0,21006.0,0.70,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
24,DABUR,735.00,-2.58,64.0,H-MC,3.94,211946.0,14918.0,74711.0,-0.21,7.57,35.25,45.49,102.0,0.20,1.57,21.60,XY24,BTT,FMCG
18,CAMS,5211.76,-6.49,47.0,H-SC,5.62,106428.0,4424.0,39506.0,-0.03,4.34,37.12,43.06,122.0,0.11,0.79,22.20,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.45,187215.0,36302.0,21006.0,0.70,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
52,NATIONALUM,247.44,-45.32,71.0,H-MC,0.47,109816.0,9312.0,21326.0,3.33,9.27,19.42,30.49,79.0,0.44,0.81,44.67,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-40.41,58.0,M-SC,4.91,100748.0,9250.0,69506.0,0.72,10.11,68.99,86.07,223.0,0.13,0.75,46.30,XR,NTT,DURABLES
39,INDIAMART,4850.92,-52.03,43.0,H-SC,12.49,133995.0,10659.0,123101.0,-0.78,8.64,91.87,108.45,119.0,0.09,0.99,32.93,AR,ATH,MISC
89,WIPRO,420.00,-11.98,50.0,M-LC,7.08,154746.0,3801.0,105661.0,-0.53,2.52,68.28,72.51,53.0,0.04,1.15,8.69,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SIS,528.00,2244.04,55.0,H-SC,5.74,63070.0,-18020.0,26691.0,1.42,-22.22,42.32,10.69,156.0,-0.68,0.47,27.82,OX40N,NTT,MISC
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,16.23,90666.0,-10297.0,104828.0,2.27,-10.20,115.62,93.63,148.0,-0.10,0.67,35.04,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,33.29,91949.0,-21600.0,61652.0,0.44,-19.02,67.05,35.27,268.0,-0.35,0.68,125.05,XR,NTT,HOTELS
51,MASFIN,398.61,-17.34,54.0,H-SC,13.25,93855.0,-4125.0,25726.0,1.92,-4.21,27.41,22.05,152.0,-0.16,0.70,36.43,XR,ATH,FINANCE
52,NATIONALUM,247.44,-45.32,71.0,H-MC,0.47,109816.0,9312.0,21326.0,3.33,9.27,19.42,30.49,79.0,0.44,0.81,44.67,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,33.29,91949.0,-21600.0,61652.0,0.44,-19.02,67.05,35.27,268.0,-0.35,0.68,125.05,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.45,187215.0,36302.0,21006.0,0.70,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
58,RELAXO,1176.00,-30.97,63.0,H-SC,6.30,64026.0,-51512.0,86506.0,2.80,-44.58,135.11,30.28,136.0,-0.60,0.47,25.87,X40N,NTT,FOOTWEAR
27,DMART,5201.00,-6.62,70.0,H-LC,14.38,233225.0,48744.0,21620.0,1.70,26.42,9.27,38.14,38.0,2.25,1.73,39.83,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,13.61,67.0,H-MC,3.24,196240.0,31570.0,36403.0,1.14,19.17,18.55,41.28,89.0,0.87,1.45,28.68,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,43.42
2,50,73.44


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.44
SC    37.16
MC    25.40
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.23
X40      15.39
X40N     14.24
XY25     11.48
AR        9.35
XR        8.97
OX40N     7.35
MH        1.74
X200      1.73
X5K       1.43
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.26
H-MC    22.42
H-SC    22.27
M-SC    13.30
M-LC     7.11
M-MC     2.66
L-SC     1.59
L-LC     1.07
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,1.35,30.79
IT,13.28,-15.23,72.38
FINANCE,7.33,-21.88,66.82
BANKS,7.28,-15.78,64.94
PAINTS,5.97,-14.52,40.16
MISC,5.81,-17.26,84.48
ELECTRICAL,5.66,-7.70,67.06
AUTO,4.59,-9.01,56.43
AC,3.77,4.95,27.25


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2787327.0,22
AR,1216586.0,10
XR,1082822.0,13
X40,889916.0,10
X40N,509206.0,11
OX40N,465710.0,10
XY25,430734.0,8
SR,196684.0,2
X5K,124112.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2768891.0,27
M-SC,1449798.0,17
H-LC,1397611.0,17
H-MC,1176228.0,17
M-LC,417740.0,5
M-MC,319503.0,2
L-SC,246869.0,3
L-MC,58854.0,1
L-LC,40145.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,925999.0,6
M-SC,XY24,862481.0,7
H-SC,AR,666467.0,5
H-LC,X40,591745.0,5
H-SC,XR,524614.0,6
H-MC,XY24,453726.0,4
H-LC,AR,369743.0,2
M-MC,XY24,319503.0,2
H-MC,X40,220420.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
